In [33]:
import cv2
import numpy as np

def detect_human(image_path):
    # Load YOLOv3 model files
    config_path = r'D:\University\Computer Vision\Project\yolov3.cfg'
    weights_path = r'D:\University\Computer Vision\Project\yolov3.weights'
    class_path = r'D:\University\Computer Vision\Project\coco.names'

    # Load the network and classes
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    classes = []
    with open(class_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Perform blob preprocessing
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set the input blob for the network
    net.setInput(blob)

    # Run forward pass and get output layers
    layer_names = net.getLayerNames()
    output_layers_names = net.getUnconnectedOutLayersNames()
    outputs = net.forward(output_layers_names)

    # Initialize lists for bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []

    # Process each output layer
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5 and class_id == 0:  # Assuming class 0 represents humans
                # Scale the bounding box coordinates
                box = detection[0:4] * np.array([width, height, width, height])
                (center_x, center_y, bbox_width, bbox_height) = box.astype("int")

                # Calculate top-left corner coordinates
                x = int(center_x - (bbox_width / 2))
                y = int(center_y - (bbox_height / 2))

                # Append the bounding box information
                boxes.append([x, y, int(bbox_width), int(bbox_height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.3)

    # Draw bounding boxes and labels for the detected humans
    for i in indices:
        i = i
        x, y, w, h = boxes[i]
        label = f"Human: {confidences[i]:.2f}"
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the image with detections
    cv2.imshow('Human Detection', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = r'D:\University\Computer Vision\Project\800px-Europei_di_pallavolo_2005_-_Italia-Russia.jpg'
detect_human(image_path)

In [4]:
import cv2
import numpy as np
import time

def detect_human_video(frame_rate):
    config_path = r'D:\University\Computer Vision\Project\yolov3.cfg'
    weights_path = r'D:\University\Computer Vision\Project\yolov3.weights'
    class_path = r'D:\University\Computer Vision\Project\coco.names'

    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    classes = []
    with open(class_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    cap = cv2.VideoCapture(r'D:\University\Computer Vision\Project\2.mp4')   #r'D:\University\Computer Vision\Project\1.mp4'

    actual_frame_rate = cap.get(cv2.CAP_PROP_FPS)

    delay = int(1000 / frame_rate)
    confidences = []
    class_ids = []
    boxes = []
    while True:
        ret, frame = cap.read()
        frame = cv2.resize(frame, (1600,800), interpolation = cv2.INTER_AREA)
        if not ret:
            break

        height, width, _ = frame.shape

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

        net.setInput(blob)

        layer_names = net.getLayerNames()
        output_layers_names = net.getUnconnectedOutLayersNames()
        outputs = net.forward(output_layers_names)

        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.8 and class_id == 0:  
                    box = detection[0:4] * np.array([width, height, width, height])
                    (x, y, w, h) = box.astype("int")
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
                    cv2.rectangle(frame, (x- int(w/2), y- int(h/2)), (x + int(w/2), y + int(h/2)), (0, 255, 0), 2)
                    label = f"Human: {confidence:.2f}"
                  #  cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    

        # for i in range(len(boxes)):
        #     if i in indexes:
        #         x, y, w, h = boxes[i]
        #         roi = frame[y- int(h/2) : y + int(h/2) , x- int(w/2) : x + int(w/2)]  # Extract the ROI
                
        #         # Save the ROI image
        #         cv2.imwrite(r'D:\University\Computer Vision\Project\Datasets\All\human_'+str(i+10940)+'.jpg', roi)
                

        cv2.imshow('Human Detection', frame)
        time.sleep(1)
        key = cv2.waitKey(delay)

        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

frame_rate = 1

detect_human_video(frame_rate)


# Classiacal

In [ ]:
import cv2
import numpy as np
import openpose

# Load OpenPose model
net = cv2.dnn.readNetFromTensorflow("pose/coco/pose_iter_440000.caffemodel", "pose/coco/pose_deploy_linevec.prototxt")

# Set input size
width, height = 368, 368

# Load input image
image = cv2.imread("input_image.jpg")
image_height, image_width, _ = image.shape

# Resize image to input size
input_image = cv2.resize(image, (width, height))

# Create a blob from the image
blob = cv2.dnn.blobFromImage(input_image, 1.0, (width, height), (127.5, 127.5, 127.5), swapRB=True, crop=False)

# Set the blob as input to the network
net.setInput(blob)

# Forward pass through the network
output = net.forward()

# Get the points and lines for each body part
points = []
lines = []
for i in range(len(openpose.BODY_PARTS)):
    # Get confidence map for body part
    prob_map = output[0, i, :, :]

    # Find global maxima of the prob_map
    _, confidence, _, point = cv2.minMaxLoc(prob_map)

    # Scale the point to the size of the input image
    x = int((image_width * point[0]) / width)
    y = int((image_height * point[1]) / height)

    # Add the point to the list
    points.append((x, y))

    # Add line indices if available
    if i in openpose.POSE_PAIRS:
        line = openpose.POSE_PAIRS[i]
        lines.append([points[line[0]], points[line[1]]])

# Draw points and lines on the image
for point in points:
    cv2.circle(image, point, 5, (0, 255, 0), -1)

for line in lines:
    cv2.line(image, line[0], line[1], (0, 0, 255), 3)

# Display the output image
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np

# Load input image
image = cv2.imread(r"D:\University\Computer Vision\Project\Datasets\All\human_1353.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# Perform edge detection
edges = cv2.Canny(blur, 120, 130)

# Find contours in the edge map
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by area in descending order
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# Iterate over contours
for contour in contours:
    # Calculate the perimeter of the contour
    perimeter = cv2.arcLength(contour, True)

    # Approximate the contour as a polygon
    epsilon = 0.02 * perimeter
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # If the contour has 4 points, it could be a human pose
    if len(approx) == 4:
        # Draw the contour on the image
        cv2.drawContours(image, [approx], -1, (0, 255, 0), 3)
        break  # Stop iterating over contours after finding the pose

# Display the output image
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np

# Load the pre-trained model
net = cv2.dnn.readNetFromCaffe('path/to/pose_deploy_linevec.prototxt', 'path/to/pose_iter_XXXX.caffemodel')

# Define the body part connections
body_parts_connections = [
    # Torso
    [0, 1], [1, 2], [2, 3], [3, 4],  # Right side
    [1, 5], [5, 6], [6, 7], [1, 8],  # Left side
    [8, 9], [9, 10], [1, 11], [11, 12],  # Hips
    [12, 13],  # Right leg
    [13, 14], [14, 15],  # Right foot
    [11, 16], [16, 17],  # Left leg
    [17, 18], [18, 19],  # Left foot
    # Arms
    [2, 16], [3, 15], [4, 14],  # Right arm
    [5, 11], [6, 12], [7, 13]  # Left arm
]

# Open the webcam or video stream
cap = cv2.VideoCapture(0)  # Change the argument to a video file path if you want to use a video file

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    # Resize the frame to a suitable size for processing
    width, height = 368, 368
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (width, height), (0, 0, 0), swapRB=True, crop=False)

    # Set the input blob as the network input
    net.setInput(input_blob)

    # Forward pass through the network
    output = net.forward()

    # Extract the detected keypoints
    num_points = output.shape[2]
    keypoints = []
    for i in range(num_points):
        # Confidence map of corresponding body's part
        prob_map = output[0, i, :, :]
        # Find the location of the body part with the highest confidence
        _, confidence, _, point = cv2.minMaxLoc(prob_map)
        # Scale the point back to the original frame size
        x = int((frame.shape[1] * point[0]) / width)
        y = int((frame.shape[0] * point[1]) / height)
        # Add the keypoint to the list
        keypoints.append((x, y) if confidence > 0.1 else None)

    # Draw the detected keypoints on the frame
    for keypoint in keypoints:
        if keypoint is not None:
            cv2.circle(frame, keypoint, 4, (0, 255, 0), -1)

    # Draw the body part connections
    for i, (a, b) in enumerate(body_parts_connections):
        if keypoints[a] is not None and keypoints[b] is not None:
            cv2.line(frame, keypoints[a], keypoints[b], (0, 255, 255), 2)

    # Display the frame
    cv2.imshow("Real-time Pose Estimation", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close any open windows
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "path/to/pose_deploy_linevec.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'


In [6]:
import cv2
import numpy as np

# Load the pre-trained HOG model for human detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open a video capture object (0 for webcam)
cap = cv2.VideoCapture(0)

while True:
    # Read the frame from the video capture
    ret, frame = cap.read()
    
    # Resize the frame to a reasonable size for faster processing
    frame = cv2.resize(frame, (640, 480))
    
    # Detect people in the frame using HOG
    boxes, weights = hog.detectMultiScale(frame, winStride=(4, 4), padding=(8, 8), scale=1.05)
    
    # Draw bounding boxes around the detected people
    for (x, y, w, h) in boxes:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Display the frame with bounding boxes
    cv2.imshow('Pose Detection', frame)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 